# 02 - Exploration du catalogue Mediatech

**Objectif** : Charger et explorer le dataset Mediatech (catalogue data.gouv vectorisé)

**Dataset** : https://huggingface.co/datasets/AgentPublic/data-gouv-datasets-catalog

**Résumé découvert** :
- 99,245 datasets (ratio 1:1, pas de chunking)
- 40 colonnes de métadonnées
- Embeddings BGE-M3 (1024 dim) stockés en JSON string

## 1. Chargement des données

In [ ]:
import duckdb
import json
import os

# Chemin vers les fichiers Parquet téléchargés depuis HuggingFace
# 2 fichiers : data_gouv_datasets_catalog_part_0.parquet et part_1.parquet
PARQUET_GLOB = "../huggingface/data_gouv_datasets_catalog_part_*.parquet"

# Connexion DuckDB en mémoire
con = duckdb.connect()

# Créer une vue pour faciliter les requêtes
con.execute(f"""
    CREATE VIEW mediatech AS 
    SELECT * FROM read_parquet('{PARQUET_GLOB}')
""")

print("✅ Vue 'mediatech' créée")

## 2. Schéma complet (40 colonnes)

In [ ]:
# Afficher le schéma
schema = con.execute("DESCRIBE mediatech").df()
print(f"📋 {len(schema)} colonnes :")
schema

In [ ]:
# Nombre de datasets
count = con.execute("SELECT COUNT(*) FROM mediatech").fetchone()[0]
print(f"📊 Nombre total de datasets : {count:,}")

## 3. Aperçu des données

In [ ]:
# Aperçu des 5 premiers datasets (colonnes clés)
preview = con.execute("""
    SELECT 
        doc_id,
        title,
        organization,
        quality_score,
        metric_views
    FROM mediatech 
    LIMIT 5
""").df()

print("👀 Aperçu :")
preview

## 4. Statistiques sur le catalogue

In [ ]:
# Top 10 organisations
top_orgs = con.execute("""
    SELECT 
        organization,
        COUNT(*) as nb_datasets,
        ROUND(AVG(quality_score), 2) as avg_quality
    FROM mediatech
    WHERE organization IS NOT NULL
    GROUP BY organization
    ORDER BY nb_datasets DESC
    LIMIT 10
""").df()

print("🏢 Top 10 organisations :")
top_orgs

In [ ]:
# Top datasets par popularité
top_views = con.execute("""
    SELECT 
        title,
        organization,
        metric_views,
        metric_reuses,
        quality_score
    FROM mediatech
    WHERE metric_views IS NOT NULL
    ORDER BY metric_views DESC
    LIMIT 10
""").df()

print("🔥 Top 10 datasets les plus vus :")
top_views

In [ ]:
# Distribution des licences
licenses = con.execute("""
    SELECT 
        license,
        COUNT(*) as nb
    FROM mediatech
    WHERE license IS NOT NULL
    GROUP BY license
    ORDER BY nb DESC
    LIMIT 10
""").df()

print("📜 Distribution des licences :")
licenses

In [ ]:
# Formats de ressources les plus courants
# Note : resources_formats est une string, on regarde les patterns
formats = con.execute("""
    SELECT 
        resources_formats,
        COUNT(*) as nb
    FROM mediatech
    WHERE resources_formats IS NOT NULL
    GROUP BY resources_formats
    ORDER BY nb DESC
    LIMIT 10
""").df()

print("📦 Top formats de ressources :")
formats

## 5. Vérification des embeddings BGE-M3

In [ ]:
# Les embeddings sont stockés en JSON string, il faut les parser
EMB_COL = "embeddings_bge-m3"

# Couverture des embeddings
coverage = con.execute(f"""
    SELECT 
        COUNT(*) as total,
        COUNT("{EMB_COL}") as avec_embedding,
        ROUND(100.0 * COUNT("{EMB_COL}") / COUNT(*), 1) as pct
    FROM mediatech
""").df()

print("📊 Couverture des embeddings :")
coverage

In [ ]:
# Récupérer et parser un embedding
sample = con.execute(f"""
    SELECT "{EMB_COL}" as emb
    FROM mediatech
    WHERE "{EMB_COL}" IS NOT NULL
    LIMIT 1
""").fetchone()[0]

print(f"Type brut : {type(sample).__name__}")
print(f"Premiers 80 chars : {sample[:80]}...")

# Parser le JSON
embedding = json.loads(sample)
print(f"\n✅ Après json.loads() :")
print(f"   Type : {type(embedding).__name__}")
print(f"   Dimension : {len(embedding)}")
print(f"   Premiers éléments : {embedding[:5]}")

In [ ]:
# Vérifier que tous les embeddings ont la même dimension
import numpy as np

# Échantillon de 100
samples = con.execute(f"""
    SELECT "{EMB_COL}" as emb
    FROM mediatech
    WHERE "{EMB_COL}" IS NOT NULL
    USING SAMPLE 100
""").fetchall()

dims = [len(json.loads(s[0])) for s in samples]
print(f"📐 Dimensions : min={min(dims)}, max={max(dims)}, unique={len(set(dims))}")

if len(set(dims)) == 1:
    print(f"✅ Tous les embeddings font {dims[0]} dimensions")

## 6. Aperçu du chunk_text (ce qui a été embedé)

In [ ]:
# Voir ce que contient chunk_text
chunk_sample = con.execute("""
    SELECT 
        title,
        chunk_text
    FROM mediatech
    WHERE chunk_text IS NOT NULL
    LIMIT 3
""").df()

for idx, row in chunk_sample.iterrows():
    print(f"\n📄 {row['title'][:60]}...")
    print(f"   chunk_text ({len(row['chunk_text'])} chars) :")
    print(f"   {row['chunk_text'][:200]}...")

## 7. Résumé

In [ ]:
nb_datasets = con.execute("SELECT COUNT(*) FROM mediatech").fetchone()[0]
nb_with_emb = con.execute(f'SELECT COUNT("{EMB_COL}") FROM mediatech').fetchone()[0]
nb_orgs = con.execute("SELECT COUNT(DISTINCT organization) FROM mediatech").fetchone()[0]

print("="*60)
print("📋 RÉSUMÉ MEDIATECH")
print("="*60)
print(f"• Datasets : {nb_datasets:,}")
print(f"• Organisations : {nb_orgs:,}")
print(f"• Colonnes : 40")
print(f"• Embeddings : {nb_with_emb:,} ({100*nb_with_emb/nb_datasets:.1f}%)")
print(f"• Dimension : 1024 (BGE-M3)")
print(f"• Format embeddings : JSON string → json.loads()")
print("="*60)
print("\n✅ Prêt pour la conversation 3 : Recherche vectorielle avec Albert API")

---

## Prochaine étape

**Conversation 3** : Implémenter la recherche sémantique
1. Appeler Albert API pour générer l'embedding d'une question
2. Calculer la similarité cosinus avec les embeddings Mediatech
3. Retourner les top-k datasets pertinents